<a href="https://colab.research.google.com/github/AG-2002/Customer-Segmentation-and-Prediction-of-Term-Deposit-Subscriptions-in-Bank-Marketing/blob/main/Bank_Data(Model_fitting(Boosting_Models))_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('bank_data_cleaned.csv')
df

,age,month,day_of_week,duration,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,poutcome_nonexistent,poutcome_success,pdays_new_recently contacted,pdays_new_sometimes ago,y
0,56,5,1,261,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
1,57,5,1,149,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
2,37,5,1,226,1,0,1.1,93.994,-36.4,4.857,...,0,1,0,0,1,1,0,0,0,0
3,40,5,1,151,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,1,0,0,0,0
4,56,5,1,307,1,0,1.1,93.994,-36.4,4.857,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41171,73,11,5,334,1,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,1
41172,46,11,5,383,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,0
41173,56,11,5,189,2,0,-1.1,94.767,-50.8,1.028,...,0,1,0,0,0,1,0,0,0,0
41174,44,11,5,442,1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,0,1


# **----------  ADA BOOST  ----------**

In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.6 MB/s eta 0:00:00


In [32]:
import optuna
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_val_score

In [8]:
X = df.drop(columns='y')
y = df['y']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
base_estimator = DecisionTreeClassifier(max_depth=1)

ada_model = AdaBoostClassifier(
    estimator=base_estimator,
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)

ada_model.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=100, random_state=42)

In [13]:
# Predict on test data
y_pred = ada_model.predict(X_test)

# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.9048081593006314
[[7096  212]
 [ 572  356]]

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      7308
           1       0.63      0.38      0.48       928

    accuracy                           0.90      8236
   macro avg       0.78      0.68      0.71      8236
weighted avg       0.89      0.90      0.89      8236



In [34]:
def objective_adaboost(trial):

    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    warm_start = trial.suggest_categorical('warm_start', [True, False])


    model = EasyEnsembleClassifier(
        n_estimators=n_estimators,
        warm_start=warm_start,
        random_state=42
    )

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='recall_macro').mean()
    return score

In [26]:
study_adaboost= optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study_adaboost.optimize(objective_adaboost, n_trials=50)

print(f'Best trial accuracy: {study_adaboost.best_trial.value}')
print(f'Best hyperparameters: {study_adaboost.best_trial.params}')

[I 2025-07-28 17:24:14,069] A new study created in memory with name: no-name-1d1f5f22-cab1-4056-9c01-7c2a8828bc96
[I 2025-07-28 17:25:27,584] Trial 0 finished with value: 0.8703290453584198 and parameters: {'n_estimators': 34, 'warm_start': True}. Best is trial 0 with value: 0.8703290453584198.
[I 2025-07-28 17:26:47,100] Trial 1 finished with value: 0.8707673909925857 and parameters: {'n_estimators': 37, 'warm_start': False}. Best is trial 1 with value: 0.8707673909925857.
[I 2025-07-28 17:29:27,816] Trial 2 finished with value: 0.8716231809089268 and parameters: {'n_estimators': 76, 'warm_start': True}. Best is trial 2 with value: 0.8716231809089268.
[I 2025-07-28 17:32:07,400] Trial 3 finished with value: 0.871505700046771 and parameters: {'n_estimators': 75, 'warm_start': True}. Best is trial 2 with value: 0.8716231809089268.
[I 2025-07-28 17:34:16,073] Trial 4 finished with value: 0.8712850407423588 and parameters: {'n_estimators': 60, 'warm_start': False}. Best is trial 2 with va

Best trial accuracy: 0.871640105234766
Best hyperparameters: {'n_estimators': 63, 'warm_start': False}


In [38]:
best_model_adaboost = EasyEnsembleClassifier(n_estimators=27, random_state=42,warm_start=False)
best_model_adaboost.fit(X_train, y_train)
y_pred = best_model_adaboost.predict(X_test)

In [40]:
# Accuracy and full classification report
print(" Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

 Accuracy: 0.8590335114133074
[[6236 1072]
 [  89  839]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.85      0.91      7308
           1       0.44      0.90      0.59       928

    accuracy                           0.86      8236
   macro avg       0.71      0.88      0.75      8236
weighted avg       0.92      0.86      0.88      8236

